1. [USDA CropScape data](https://nassgeodata.gmu.edu/CropScape/)
2. Select county AOI
3. Download county for 2017, **disable** frequency and mask, set projection to UTM 15
4. Extract and rename to `CDL_2017_<FIPS>.tif`, where `<FIPS>` is the county FIPS code (`200001` for Allen)

In [1]:
%%capture
%%script false  # disable to run cell

# Install required libraries
!pip install rasterio numpy geopandas rasterstats

In [2]:
# Load required libraries
import csv
import rasterio
from rasterio.features import shapes
from shapely.geometry import shape
import rasterstats
import numpy as np
import geopandas as gpd
from os.path import exists

In [3]:
# Function to mask any raster by any value
def mask_raster(src_path, value_to_keep, dst_path):
    '''Entire crop raster -> Masked raster'''
    with rasterio.open(src_path) as src:
        raster_data = src.read()
        mask = np.isin(raster_data, value_to_keep, invert=True)
        masked_data = np.ma.masked_array(raster_data, mask=mask)
        meta = src.meta
    
    with rasterio.open(dst_path, 'w+', **meta) as dst:
        dst.write(masked_data)

In [4]:
# Function to convert crop raster to GeoDataFrame
def crop_to_gdf(src_path, fips, crop):
    '''Masked crop raster -> GeoDataFrame'''
    with rasterio.open(src_path) as src:
        raster_data = src.read(1)
        shapes_ = list(shapes(raster_data, mask=None, transform=src.transform))
    
    polygons = []
    for geom, val in shapes_:
        if val == crop:  # select only crop polygons
            polygons.append({'geometry': shape(geom)})
    
    gdf = gpd.GeoDataFrame(polygons)
    gdf = gdf.set_crs(src.crs)  # set CRS
    
    gdf['area'] = gdf.geometry.area
    
    return gdf

In [16]:
def process_crop(src_path, fips, crop, dest):
    gdf = crop_to_gdf(src_path, fips, crop)
    
    # Remove small areas
    selection = gdf.area > 2023  # 2023m2 is  0.5 acres
    gdf = gdf.loc[selection]
    
    if gdf.empty:
        raise AttributeError('No large enough polygons.')  # TODO: too lazy to pick better error
    
    # Buffer
    gdf['geometry'] = gdf.buffer(60.96)  # 0.4 mile = 60.96m
    
    gdf.to_file(dest)

### TODO: Rename all county TOFI files to `TOFI_2017_<FIPS>.tif`

In [21]:
# fipses = [20001, 20003]
fipses = range(20001, 20114, 2)
# crops = [1, 5, 6, 23, 24, 26, 225, 226, 236, 237, 238, 240, 241, 254]
crops = [1, 5, 24, 26]
# crops = [240]

prefix = 'D:\\windbreak-croploss\\'

def main():
    data = []
    
    for crop in crops:
        # TODO: extract crop raster from state file
            for fips in fipses:
                print(fips, crop)  # for debug
                try:
                    crop_file = f'{prefix}sourcedata\\CDL_2017_{fips}.tif'
                    tree_file = f'{prefix}sourcedata\\TOFI_2017_{fips}.tif'
                    crop_mask_file = f'{prefix}data\\masked_{crop}_CDL_2017_{fips}.tif'
                    tree_mask_file = f'{prefix}data\\masked_TOFI_2017_{fips}.tif'
                    shp_file = f'{prefix}data\\crop_{crop}_{fips}.shp'
                    
                    if not exists(tree_mask_file):
                        mask_raster(tree_file, 1, tree_mask_file)
                    
                    if not exists(shp_file):
                        if not exists(crop_mask_file):
                            mask_raster(crop_file, crop, crop_mask_file)
                        process_crop(crop_mask_file, fips, crop, shp_file)
                    
                    # Count tree pixels
                    # TODO: there must be a faster method than zonal_stats
                    stats = rasterstats.zonal_stats(vectors=shp_file,
                                                    raster=tree_mask_file,
                                                    stats=['sum'],
                                                    nodata=-9999,
                                                    all_touched=True)
                    
                    tree_px = sum([x['sum'] for x in stats])
                    
                    # calc crop area
                    # crop resolution is 30m, so 'area' is in m^2
                    # tree resolution is 2m
                    
                    gdf = gpd.read_file(shp_file)
                    crop_area = gdf['area'].sum()
                    
                    # gdf.plot()
                    
                    # assemble data output
                    data.append({'fips': fips, 'crop_id': crop, 'crop_area_m2': crop_area, 'tree_area_m2': tree_px*4})
                
                except AttributeError:  # crop type not in county
                    print(f'County {fips} has no crop of id {crop}.')
                    pass
    
    # write data to csv
    header = ['fips', 'crop_id', 'crop_area_m2', 'tree_area_m2']
    with open('output.csv', 'w', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=header)
        writer.writeheader()
        writer.writerows(data)

In [ ]:
import cProfile

cProfile.run('main()')

20001 1
20003 1
20005 1
20007 1
20009 1
20011 1
20013 1
20015 1
20017 1
20019 1
20021 1
20023 1
20025 1
20027 1
20029 1
20031 1
20033 1
20035 1
20037 1
20039 1
